# utility.py

In [1]:

# My Utility : auxiliars functions

import pandas as pd
import numpy  as np


# CResidual-Dispersion Entropy
def entropy_dispersion(x,d,tau,c):
    pass
    #...
    #return(entr)
#

# Permutation Entropy
def entropy_permuta(x,m,tau):
    pass
    #...
    #return(entr)
#




# ppr.py


In [ ]:
"""
================================================================================
REQUERIMIENTOS FUNCIONALES Y NO FUNCIONALES: ppr.py
(Etapa #1: Pre-Procesamiento de la Data)
================================================================================

--- RF-001: cargar_datos ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga Class1.csv a Class4.csv (4-muestras de N=120000 valores).             |
|                     | Notación: 𝑋 ∈ ℜ^(N,4), N=120000.                                            |
| **Entradas**        | - Rutas a Class1.csv, Class2.csv, Class3.csv, Class4.csv.                   |
| **Salidas**         | - Matriz 𝑋 ∈ ℜ^(120000,4).                                                  |
| **Parámetros**      | - rutas: Lista de rutas a archivos CSV.                                     |
| **Requerimientos NF**| [RNF-001] Validar 120000 filas por archivo.                                |
|                     | [RNF-002] Validar ausencia de valores NaN.                                  |
| **Ejemplo**         | X = cargar_datos(["Class1.csv", "Class2.csv", "Class3.csv", "Class4.csv"])  |
| **Notas**           | - Usar pandas: pd.read_csv(..., header=None).                               |

--- RF-002: aplicar_diferencia_finita ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Aplica diferencias finitas: 𝑥(n) = 𝑥(n) − 𝑥(n−1), n=2,...,N.                 |
| **Entradas**        | - Matriz 𝑋 ∈ ℜ^(120000,4).                                                  |
| **Salidas**         | - Matriz 𝑋_diff ∈ ℜ^(119999,4).                                             |
| **Parámetros**      | - X: Matriz de tamaño (120000,4).                                           |
| **Requerimientos NF**| [RNF-003] Usar np.diff(X, axis=0).                                         |
|                     | [RNF-004] Validar salida con 119999 filas.                                  |
| **Ejemplo**         | X_diff = aplicar_diferencia_finita(X)                                       |
| **Notas**           | - La primera fila se pierde al aplicar la diferencia.                       |

--- RF-003: segmentar_muestras ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Segmenta muestras con longitud 𝑙𝐹 ∈ {1000,1200,1600}.                        |
|                     | Notación: 𝑥_i ∈ ℜ^(𝑙𝐹, 𝑛𝐹), 𝑛𝐹 = N/𝑙𝐹.                                        |
| **Entradas**        | - Matriz 𝑋_diff ∈ ℜ^(119999,4).                                             |
|                     | - 𝑙𝐹: Longitud del segmento.                                                 |
| **Salidas**         | - Lista de matrices segmentadas.                                             |
| **Parámetros**      | - X_diff: Matriz de tamaño (119999,4).                                       |
|                     | - lF: Longitud del segmento (1000, 1200, o 1600).                            |
| **Requerimientos NF**| [RNF-005] Validar que 119999 sea divisible por 𝑙𝐹.                           |
|                     | [RNF-006] Segmentar por filas, sin solapamiento.                             |
| **Ejemplo**         | segmentos = segmentar_muestras(X_diff, lF=1000)                              |
| **Notas**           | - Usar np.reshape.                                                           |

--- RF-004: calcular_entropias ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Calcula entropías multi-escala (MDE, eMDE, MPE, eMPE) por segmento.         |
| **Entradas**        | - Lista de segmentos.                                                       |
|                     | - 𝑑, 𝜏, 𝑐, 𝑆_max: Parámetros de entropía.                                    |
| **Salidas**         | - DataFrame de entropías concatenadas (dClases.csv).                        |
| **Parámetros**      | - segmentos: Lista de matrices segmentadas.                                 |
|                     | - d: Dimensión embedding.                                                   |
|                     | - tau: Factor de retardo.                                                   |
|                     | - c: Número de símbolos.                                                    |
|                     | - Smax: Número máximo de escalas.                                           |
| **Requerimientos NF**| [RNF-007] Implementar fórmulas según slides del profesor.                  |
|                     | [RNF-008] Validar parámetros enteros positivos.                             |
| **Ejemplo**         | dClases = calcular_entropias(segmentos, d=2, tau=1, c=3, Smax=10)           |
| **Notas**           | - Concatenar los 4 datasets de entropías.                                   |

--- RF-005: generar_etiquetas ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Crea etiquetas binarias para 4 clases: (1 0 0 0), (0 1 0 0), etc.           |
| **Entradas**        | - Número de muestras por clase.                                             |
| **Salidas**         | - DataFrame de etiquetas (dLabel.csv).                                      |
| **Parámetros**      | - n_muestras_por_clase: Número de muestras.                                 |
| **Requerimientos NF**| [RNF-009] Validar coincidencia de filas con segmentos.                     |
| **Ejemplo**         | dLabel = generar_etiquetas(n_muestras_por_clase=30000)                      |
| **Notas**           | - Usar one-hot encoding.                                                    |

--- RF-006: guardar_configuracion ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Guarda configuración en conf_ppr.csv (6 líneas).                            |
| **Entradas**        | - tipo_entropia, lF, d, tau, c, Smax.                                       |
| **Salidas**         | - Archivo conf_ppr.csv.                                                     |
| **Parámetros**      | - tipo_entropia: 1-4 (MDE, eMDE, MPE, eMPE).                                |
|                     | - lF, d, tau, c, Smax: Parámetros numéricos.                                |
| **Requerimientos NF**| [RNF-010] Validar parámetros enteros positivos.                            |
| **Ejemplo**         | guardar_configuracion(1, 1000, 2, 1, 3, 10)                                 |
| **Notas**           | - Formato: 1 línea por parámetro.                                           |
"""





#----------------------------------------------
# Create Features by use Multi-scales Entropy
#----------------------------------------------

import pandas  as pd
import numpy   as np
from utility   import *


    
# Obtain Features by use Entropy    
def gets_features():
    pass
    #return(F)    
# Beginning ...


def main():
    pass
    
       
if __name__ == '__main__':   
	 main()



# trn.py

In [ ]:
"""
================================================================================
REQUERIMIENTOS FUNCIONALES Y NO FUNCIONALES: trn.py
(Etapa #2: Algoritmo de Entrenamiento)
================================================================================

--- RF-001: cargar_datos ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga dClases.csv y dLabel.csv.                                             |
| **Entradas**        | - Rutas a dClases.csv y dLabel.csv.                                         |
| **Salidas**         | - Matrices 𝑋 (características) y 𝑦 (etiquetas).                             |
| **Parámetros**      | - ruta_X: Ruta a dClases.csv.                                               |
|                     | - ruta_y: Ruta a dLabel.csv.                                                |
| **Requerimientos NF**| [RNF-001] Validar mismo número de filas en 𝑋 y 𝑦.                          |
| **Ejemplo**         | X, y = cargar_datos("dClases.csv", "dLabel.csv")                            |

--- RF-002: reordenar_aleatoriamente ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Reordena aleatoriamente 𝑋 y 𝑦 sincronizadamente.                            |
| **Entradas**        | - Matrices 𝑋 y 𝑦.                                                           |
| **Salidas**         | - Matrices 𝑋_shuffled y 𝑦_shuffled.                                         |
| **Parámetros**      | - seed: Semilla para reproducibilidad (ej. 42).                             |
| **Requerimientos NF**| [RNF-002] Usar np.random.seed(42).                                         |
|                     | [RNF-003] Validar orden sincronizado.                                       |
| **Ejemplo**         | X_shuffled, y_shuffled = reordenar_aleatoriamente(X, y)                     |

--- RF-003: normalizar_zscore ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Normaliza características: 𝑥 = (𝑥 − mean(𝑥)) / std(𝑥).                       |
| **Entradas**        | - Matriz 𝑋.                                                                 |
| **Salidas**         | - Matriz 𝑋_norm normalizada.                                                |
| **Parámetros**      | - X: Matriz de características.                                             |
| **Requerimientos NF**| [RNF-004] Validar ausencia de divisiones por cero.                         |
| **Ejemplo**         | X_norm = normalizar_zscore(X)                                               |

--- RF-004: dividir_train_test ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Divide datos en entrenamiento (65-80%) y prueba.                            |
| **Entradas**        | - 𝑋_norm, 𝑦_shuffled, porcentaje 𝑝 (65 < 𝑝 < 81).                            |
| **Salidas**         | - 𝑋_train, 𝑋_test, 𝑦_train, 𝑦_test.                                          |
| **Parámetros**      | - p: Porcentaje para entrenamiento (ej. 0.75).                              |
| **Requerimientos NF**| [RNF-005] Validar 𝑝 ∈ [0.65, 0.80].                                        |
| **Ejemplo**         | X_train, X_test, y_train, y_test = dividir_train_test(X_norm, y_shuffled, 0.75)|

--- RF-005: entrenar_mgd ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Entrena modelo con mGD (descenso de gradiente con momentum).                |
| **Entradas**        | - 𝑋_train, 𝑦_train, max_iter, 𝜇, momentum.                                  |
| **Salidas**         | - Matriz de pesos 𝑊, vector de costo 𝐽.                                     |
| **Parámetros**      | - max_iter: Máximo de iteraciones.                                          |
|                     | - mu: Tasa de aprendizaje.                                                  |
|                     | - momentum: Factor de momentum.                                             |
| **Requerimientos NF**| [RNF-006] Inicializar 𝑊 aleatoriamente.                                    |
|                     | [RNF-007] Validar disminución monótona del costo.                           |
| **Ejemplo**         | W, J = entrenar_mgd(X_train, y_train, 1000, 0.01, 0.9)                      |

--- RF-006: guardar_pesos_y_costo ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Guarda pesos y costos en pesos.csv y costo.csv.                             |
| **Entradas**        | - 𝑊, 𝐽, rutas de salida.                                                    |
| **Salidas**         | - Archivos pesos.csv y costo.csv.                                           |
| **Parámetros**      | - ruta_pesos: Ruta a pesos.csv.                                             |
|                     | - ruta_costo: Ruta a costo.csv.                                             |
| **Requerimientos NF**| [RNF-008] Validar dimensiones de 𝑊 y 𝐽.                                    |
| **Ejemplo**         | guardar_pesos_y_costo(W, J, "pesos.csv", "costo.csv")                       |

--- RF-007: cargar_configuracion ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga configuración desde conf_train.csv.                                   |
| **Entradas**        | - Ruta a conf_train.csv.                                                    |
| **Salidas**         | - max_iter, 𝜇, 𝑝.                                                           |
| **Parámetros**      | - ruta: Ruta al archivo.                                                    |
| **Requerimientos NF**| [RNF-009] Validar valores numéricos positivos.                             |
| **Ejemplo**         | max_iter, mu, p = cargar_configuracion("conf_train.csv")                    |
"""



# Softmac Regression's Training :

import numpy  as np
from utility  import *


#
#Training by use mGD
def train():    
    
    pass
    #....
    #return()

# Beginning ...
def main():    
    pass
       
if __name__ == '__main__':   
	 main()



# test.py

In [ ]:
"""
================================================================================
REQUERIMIENTOS FUNCIONALES Y NO FUNCIONALES: tst.py
(Etapa #3: Evaluación de Rendimiento)
================================================================================

--- RF-001: cargar_datos ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Carga dtst.csv, dtst_label.csv y pesos.csv.                                 |
| **Entradas**        | - Rutas a los archivos.                                                     |
| **Salidas**         | - 𝑋_test, 𝑦_test, 𝑊.                                                        |
| **Parámetros**      | - ruta_X_test, ruta_y_test, ruta_W.                                         |
| **Requerimientos NF**| [RNF-001] Validar compatibilidad de dimensiones.                           |
| **Ejemplo**         | X_test, y_test, W = cargar_datos("dtst.csv", "dtst_label.csv", "pesos.csv") |

--- RF-002: predecir_softmax ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Calcula probabilidades con regresión softmax.                               |
| **Entradas**        | - 𝑋_test, 𝑊.                                                               |
| **Salidas**         | - Matriz de probabilidades 𝑦_pred_proba.                                    |
| **Parámetros**      | - X_test: Matriz de características.                                        |
|                     | - W: Matriz de pesos.                                                       |
| **Requerimientos NF**| [RNF-002] Implementar softmax manualmente.                                 |
|                     | [RNF-003] Validar ausencia de overflow.                                     |
| **Ejemplo**         | y_pred_proba = predecir_softmax(X_test, W)                                  |

--- RF-003: obtener_etiquetas_predichas ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Obtiene etiquetas predichas desde probabilidades.                           |
| **Entradas**        | - 𝑦_pred_proba.                                                             |
| **Salidas**         | - Vector de etiquetas 𝑦_pred.                                               |
| **Parámetros**      | - y_pred_proba: Matriz de probabilidades.                                   |
| **Requerimientos NF**| [RNF-004] Usar np.argmax.                                                  |
| **Ejemplo**         | y_pred = obtener_etiquetas_predichas(y_pred_proba)                          |

--- RF-004: matriz_confusion ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Calcula matriz de confusión (4x4).                                          |
| **Entradas**        | - 𝑦_test, 𝑦_pred.                                                           |
| **Salidas**         | - Matriz de confusión 𝑐𝑚 ∈ ℜ^(4,4).                                         |
| **Parámetros**      | - y_test: Etiquetas reales.                                                 |
|                     | - y_pred: Etiquetas predichas.                                              |
| **Requerimientos NF**| [RNF-005] Inicializar matriz de ceros.                                     |
| **Ejemplo**         | cm = matriz_confusion(y_test, y_pred)                                       |

--- RF-005: calcular_fscores ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Calcula F-scores por clase: 𝐹1 = 2 × (precision × recall) / (precision + recall). |
| **Entradas**        | - Matriz de confusión 𝑐𝑚.                                                   |
| **Salidas**         | - Vector de F-scores 𝑓𝑠𝑐𝑜𝑟𝑒𝑠 ∈ ℜ^(1,4).                                       |
| **Parámetros**      | - cm: Matriz de confusión.                                                  |
| **Requerimientos NF**| [RNF-006] Calcular precision y recall por clase.                           |
| **Ejemplo**         | fscores = calcular_fscores(cm)                                              |

--- RF-006: guardar_resultados ---
| Campo               | Descripción                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Descripción**     | Guarda resultados en cmatriz.csv y fscores.csv.                             |
| **Entradas**        | - 𝑐𝑚, 𝑓𝑠𝑐𝑜𝑟𝑒𝑠, rutas de salida.                                               |
| **Salidas**         | - Archivos cmatriz.csv y fscores.csv.                                       |
| **Parámetros**      | - ruta_cm: Ruta a cmatriz.csv.                                              |
|                     | - ruta_fs: Ruta a fscores.csv.                                              |
| **Requerimientos NF**| [RNF-007] Validar dimensiones de 𝑐𝑚 y 𝑓𝑠𝑐𝑜𝑟𝑒𝑠.                               |
| **Ejemplo**         | guardar_resultados(cm, fscores, "cmatriz.csv", "fscores.csv")               |
"""






# Testing for Softmax Regresion
import numpy as np

# Beginning ...
def main():
    pass			

if __name__ == '__main__':   
	 main()

